# So easy, *voilà* IPyMaps!

In this example notebook, we demonstrate how voila can render custom interactive maps with built-in search.

In [3]:
import re
import os

import requests
import geojson
from ipywidgets import Text, HTML, Layout
from ipyleaflet import Map, GeoJSON, WidgetControl, FullScreenControl, LayersControl
from geopy.geocoders import Nominatim

In [4]:
class SearchMap(Map):
    """
    This widget provides a simple maps-search UI.
    
    For now this is a simple wrapper around ipyleaflet.Map mainly adding a search
    text field. Depending on the sumitted text one of multiple things can happen
    (attempted in the given order):
    
    1. lat/lon: center the map to the respective coordinates
    2. URL ending in '.geojson' or 'geo.json': load and display GeoJSON data
    3. text: geolocate address and center the map to the respective coordinates
    """
    def __init__(self, *args, geocoder=None, **kwargs):
        self.geocoder = geocoder or Nominatim(user_agent="ipymaps")
        
        super().__init__(*args, zoom_control=False, **kwargs)
        # FIXME: add zoom control again once it can be put below the search field

        self.status_control = None

        # add search text field
        self.search_tx = Text(layout=Layout(width='200px'))
        self.search_tx.on_submit(self.text_submitted)
        wc = WidgetControl(widget=self.search_tx, position='topleft')
        self.add_control(wc)
        
        # add other controls
        self.add_control(FullScreenControl())
        self.add_control(LayersControl())

    def set_status(self, text):
        self.status_tx = HTML(text, layout=Layout(width='50%'))
        self.status_control = WidgetControl(widget=self.status_tx, position='bottomleft')
        self.add_control(self.status_control)

    def reset_status(self):
        if self.status_control:
            self.remove_control(self.status_control)
            self.status_control = None

    def text_submitted(self, widget):
        self.run_query(widget.value)

    def run_query(self, text):
        # lat lon
        try:
            values = re.split('[/,;: ]', text)
            lat, lon = [v for v in values if v not in '/,;:']
            self.center = lat, lon
            return
        except ValueError:
            pass

        # geojson
        url = text.strip()
        if url.endswith('.geojson') or url.endswith('.geo.json'):
            self.set_status('Loading...')
            gj = requests.get(url).content
            self.reset_status()
            data = geojson.loads(gj)
            name = os.path.basename(url)
            self.add_layer(GeoJSON(data=data, name=name))
            return

        # simple geolocation
        loc = self.geocoder.geocode(text)
        self.center = loc.latitude, loc.longitude

In [6]:
SearchMap(center=[52, 13], zoom=4)

SearchMap(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution':…